In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from MC import *


## Task 3: Computing `g(r)` and `S(k)` of liquid Argon using Monte Carlo

Using the system described in Rahamn (the same as used in Task II for molecular
dynamics), converge the various Monte Carlo parameters and compute the radial
distribution function `g(r)` and structure factor `S(k)`.

To ensure efficient use of computation time, first equilibrate the system and
perform a block-average analysis to determine the correlation length of the system.
Use this knowledge to set `fsamp` such that you only compute `g(r)` for 
uncorrelated samples. Compare `g(r)` and `S(k)` from Monte Carlo to those obtained
in Task II from molecular dynamics and to those reported by Rahman.


In [ ]:
# Simulation parameters
npart = 864        # Number of particles
rho   = 0.8072     # Density (LJ units)

# Monte Carlo parameters
init_dmax   = 0.1       # maximal distance in one displacement (LJ units)
ndispl = 50        # number of displacement attempts in one move
nequil = 10_000    # number of equilibration cycles
lmax   = 2_000     # number of production cycles
fsamp  = 10        # frequency of sampling (number of sampling lmax/nsamp)

# Lennard-Jones parameters (NOTE: we're using Lennard-Jones units!)
eps  = 1.0  # 120.0 K in real units
sig  = 1.0  # 3.4 Å in real units
rcut = 5.0  # Cutoff radius (LJ units)


# Parameter for s(k)
qmax  = 32.0  # Max range
nqvec = 144   # Number of points

# Temperature
temp   = 0.7833    # LJ units, ~94 K in real units
beta   = 1 / temp  # kB == 1 in these LJ units!

nbins  = 2 * (100 // 2)  # sampling for g(r), made to be always even


In [ ]:
vol     = npart / rho                 # volume of the box
box     = np.cbrt(vol)                # length of the cubic lattice
rcut    = min(box / 2.0, rcut)        # Check that cutoff is not larger than the box
r2cut   = rcut**2                     # Squared cutoff raidus
new_pos = np.array([0.0, 0.0, 0.0])   # Array for the trial position

# Create initial positions
pos = ini_pos_fcc(npart, box)

_, _ = energy(pos, r2cut, eps, sig, npart, box, rho, vol, beta)


In [ ]:
# Search for the optimal maximum displacement (s.t. accepted rate = 0.5)
tot_frac = 0.5
opt_dmax = init_dmax
move_fraction_log = []
dmax_log = []
avg_fraction_log = []
for i in range(nequil):
    naccpt=0
    if (i + 1) % 5 == 0:
        # Update dmax after every 5 cycles
        dmax_old = opt_dmax
        dmax_new = dmax_old * (tot_frac / 0.5)
        # Limit change in dmax (0.5 dmax_old < dmax_new < 1.5 dmax_old)
        dmax_new = max(0.5 * dmax_old, dmax_new)
        dmax_new = min(1.5 * dmax_old, dmax_new)
        # Limit dmax (dmax <= box / 2)
        dmax_new = min(0.5 * box, dmax_new)
        opt_dmax = dmax_new

    move_frac, _, _ = move(
        pos, new_pos, opt_dmax,
        ndispl, npart, box,
        r2cut, eps, sig,
        beta, rho, vol, False
    )

    move_fraction_log.append(move_frac)
    dmax_log.append(opt_dmax)
    avg_fraction_log.append(tot_frac)

    tot_frac = np.mean(move_fraction_log)

    if (i + 1) % 500 == 0:
        print(f'cycle: {i+1: 8d}  frac. accp.: {tot_frac:0.4f}  dmax: {opt_dmax:0.4f}')

print(f'Final dmax: {opt_dmax:0.4f}')
# Write the positions of the atoms and the optimized dmax to disk
filename = 'argon_equilibrated.dat'
print(f'Writing final configuration to {filename}')
dump_pos(filename, pos, npart, box, opt_dmax)

fig, left_ax = plt.subplots()
right_ax = left_ax.twinx()

left_ax.set_ylabel('Acceptance ratio')
left_ax.set_xlabel('Equilibration step')
left_ax.set_ylim(-0.05, +1.05)
left_ax.plot(move_fraction_log, label='Move')
left_ax.plot(avg_fraction_log, label='Average')
left_ax.legend()

right_ax.set_ylabel(r'd$_{max}$')
right_ax.plot(dmax_log, c='k')


In [ ]:
pos, npart, box, dmax = read_pos('argon_equilibrated.dat')

g = np.zeros(nbins, dtype=int)
n_samples_g = 0
for i in range(lmax):
    _, _, _ = move(
        pos, new_pos, dmax,
        ndispl, npart, box,
        r2cut, eps, sig,
        beta, rho, vol, False
    )
    if (i + 1) % fsamp == 0:
        gofr(g, pos, box, rho, nbins, npart)
        n_samples_g += 1

g_average = ave_gofr(g, box, npart, nbins, n_samples_g)
s_average = sofk_FT(g_average, box, npart, qmax, nqvec)

with open('argon_gr.dat', 'w') as fp:
    fp.write('# r/sigma g(r)\n')
    for (r, g) in zip(g_average['r'], g_average['g']):
        fp.write(f'{r:0.8f} {g:0.8f}\n')

with open('argon_sk.dat', 'w') as fp:
    fp.write('# k*sigma g(r)\n')
    for (k, s) in zip(s_average['k'], s_average['s']):
        fp.write(f'{k:0.8f} {s:0.8f}')

fig, axes = plt.subplots(1, 2, figsize=(9,4))

axes[0].plot(g_average['r'], g_average['g'])
axes[0].set_xlabel('r/$\sigma$')
axes[0].set_ylabel('g(r)')

axes[1].plot(s_average['k'], s_average['s'])
axes[1].set_xlabel('k*$\sigma$')
axes[1].set_ylabel('S(k)')
